# Secretary Module - Local Interactive Testing

**Prerequisites:**
1. Run `./setup_local_env.sh` first
2. Activate environment: `source venv/bin/activate`
3. Start Ollama: `ollama serve` (in separate terminal)
4. Ensure Mistral model is pulled: `ollama pull mistral`

**Run cells in order from top to bottom!**

## 1. Environment Check

In [ ]:
import sys
import os
import subprocess
import requests

print("🔍 Environment Check")
print("="*60)

# Python version
print(f"✓ Python: {sys.version.split()[0]}")

# Check if in virtual environment
in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
print(f"{'✓' if in_venv else '⚠️ '} Virtual environment: {in_venv}")
if not in_venv:
    print("   Warning: Not in venv! Run 'source venv/bin/activate' first")

# Check GPU
try:
    import torch
    has_gpu = torch.cuda.is_available()
    print(f"{'✓' if has_gpu else 'ℹ️ '} GPU: {has_gpu}")
    if has_gpu:
        print(f"   Device: {torch.cuda.get_device_name(0)}")
    else:
        print("   Running on CPU (slower but works)")
except ImportError:
    print("ℹ️  GPU: PyTorch not installed yet (will check during TTS install)")

# Check Ollama
try:
    response = requests.get('http://localhost:11434/api/tags', timeout=2)
    models = response.json().get('models', [])
    mistral_available = any('mistral' in m['name'] for m in models)
    print(f"✓ Ollama: Running")
    print(f"{'✓' if mistral_available else '⚠️ '} Mistral: {mistral_available}")
    if not mistral_available:
        print("   Run: ollama pull mistral")
except:
    print("❌ Ollama: Not running")
    print("   Run in separate terminal: ollama serve")

print("="*60)

## 2. Import Secretary Module

In [ ]:
from secretary import Secretary, create_full_workflow
from tool_registry import AVAILABLE_TOOLS, REAL_SCRIPTWRITER_AVAILABLE, REAL_AUDIOAGENT_AVAILABLE
import json

print("✅ Secretary loaded!")
print(f"\n🔧 Worker Status:")
print(f"   ScriptWriter: {'REAL ✅' if REAL_SCRIPTWRITER_AVAILABLE else 'MOCK ⚠️'}")
print(f"   AudioAgent: {'REAL ✅' if REAL_AUDIOAGENT_AVAILABLE else 'MOCK ⚠️'}")

if not REAL_SCRIPTWRITER_AVAILABLE:
    print("\n⚠️  ScriptWriter failed to load - check Ollama is running")
if not REAL_AUDIOAGENT_AVAILABLE:
    print("\n⚠️  AudioAgent failed to load - TTS/Whisper may not be installed")
    print("   Run: pip install TTS openai-whisper")

if REAL_SCRIPTWRITER_AVAILABLE and REAL_AUDIOAGENT_AVAILABLE:
    print("\n✅ All workers ready!")

## 3. Input Requirements

Enter your video requirements interactively:

In [ ]:
print("="*60)
print("VIDEO REQUIREMENTS")
print("="*60)

topic = input("\n1. Video topic (min 5 chars): ")
print("\n2. Style: funny, documentary, serious, graphic-heavy, tutorial, or custom")
style = input("   Your choice: ")
print("\n3. Duration (e.g., '5 minutes', 'like 30 seconds', 'around 2-3 mins')")
duration = input("   Duration: ")
audio = input("\n4. Audio mode (generate/upload): ")
script = input("\n5. Script mode (generate/upload): ")
comments = input("\n6. Comments (optional, press Enter to skip): ")

requirements = {
    'topic': topic,
    'style': style,
    'duration_range': duration,
    'audio_mode': audio,
    'script_mode': script,
    'comments': comments or None
}

print("\n" + "="*60)
print(json.dumps(requirements, indent=2))

## 4. Validate Requirements

In [ ]:
sec = Secretary()
is_valid, errors = sec.validate_inputs(requirements)

print("\n" + "="*60)
print("VALIDATION RESULT")
print("="*60)

if is_valid:
    print("✅ ALL INPUTS VALID!")
    sec.set_requirements(requirements)
else:
    print("❌ VALIDATION FAILED!\n")
    for error in errors:
        print(f"  - {error}")
    print("\n⚠️  Fix errors and re-run cell 3!")

## 5. Generate Script (Ollama + LangSearch)

**Takes 30-90 seconds**

In [ ]:
print("🚀 Generating script...\n")

result = sec.call_tool('scriptwriter', sec.requirements)

if result['success']:
    print("\n" + "="*80)
    print("✅ SCRIPT GENERATED!")
    print("="*80)
    
    outputs = result['outputs']
    metadata = result.get('metadata', {})
    
    print(f"\n📊 Stats: {metadata.get('actual_words', 0)} words, ~{outputs.get('estimated_duration_minutes', 0)} min")
    
    sources = outputs.get('research_sources', [])
    if sources:
        print(f"\n📚 Research: {len(sources)} sources")
    
    print(f"\n💡 Talking points: {outputs.get('talking_points_count', 0)}")
    
    print("\n" + "="*80)
    print("📜 SCRIPT:")
    print("="*80)
    print(outputs['script'])
    print("="*80)
    
    script = outputs['script']
    print("\n✅ Ready for audio generation!")
else:
    print(f"\n❌ FAILED: {result.get('error', 'Unknown error')}")

## 6. Generate Audio (XTTS + Whisper)

**Takes 1-3 minutes (GPU) or 5-10 minutes (CPU)**

In [ ]:
print("🎤 Generating audio...\n")

style = sec.requirements.get('style', 'neutral')
result = sec.call_tool('audio_agent', {'script': script, 'style': style})

if result['success']:
    print("\n" + "="*80)
    print("✅ AUDIO GENERATED!")
    print("="*80)
    
    outputs = result['outputs']
    
    print(f"\n📊 Duration: {outputs['duration']:.1f}s ({outputs['duration']/60:.1f} min)")
    print(f"📁 File: {outputs['audio_path']}")
    print(f"💾 Size: {outputs['file_size_mb']:.2f} MB")
    
    transcript = outputs['transcript_timestamps']
    segments = transcript['segments']
    
    print(f"\n📝 Segments: {len(segments)}")
    print(f"🔇 Silence points: {len(transcript.get('silence_points', []))}")
    
    print("\n🎯 First 5 segments:")
    for i, seg in enumerate(segments[:5], 1):
        print(f"   [{seg['start']:.1f}s-{seg['end']:.1f}s] {seg['type'].upper()}: {seg['text'][:60]}...")
    
    audio_file = outputs['audio_file']
    print(f"\n✅ Audio saved to: {outputs['audio_path']}")
else:
    print(f"\n❌ FAILED: {result.get('error', 'Unknown error')}")

## 7. LangSearch (Mock)

Coming next...

In [ ]:
print("⚠️  LangSearch worker not yet implemented - using mock")

result = sec.call_tool('langsearch', {'script': script})

if result['success']:
    research_data = result['outputs']['research_data']
    print(f"✅ Mock research complete: {research_data['total_terms_found']} terms")
else:
    print(f"❌ FAILED: {result.get('error')}")

## 8. Other Workers (Mocks)

BrainBox, AssetCollector, Executor - coming soon!